# 10-714: Homework 1

In this homework you will build the basics of a reverse mode automatic differentiation engine which we will refer to as __needle__ (__ne__cessary __e__lements of __d__eep __le__arning). Before we dive into the code, let's setup the connection to Drive just as we did for homework 0, and install any necessary packages. To get started, make a copy of this notebook file by selecting "Save a copy in Drive" from the "File" menu, and then run the code block below.

In [ ]:
# # Code to set up the assignment
# from google.colab import drive
# drive.mount('/content/drive')
# #%cd /content/drive/MyDrive/
# # !mkdir -p 10714
# # %cd /content/drive/MyDrive/10714
# # TODO: UPDATE THIS
# # !git clone https://github.com/dlsys10714/hw1.git
# %cd /content/drive/MyDrive/10714/hw1

# !pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
# !pip3 install numdifftools

import sys
sys.path.append('./python')
sys.path.append('./python/apps')
from simple_ml import *

grader_key = 'YOUR_MUGRADE_KEY_HERE'

## Introduction to `needle`

The purpose of `needle` is to enable the creation and manipulation of computation graphs. Computation graphs are composed of `Tensor` objects, each of which having an operation and a value. Furthermore, in future homeworks we will extend what we are developing here to work with multiple hardware backends (e.g. cuda). We have provided you with these base classes, and understanding them will be critical to this homework as you will use this as a starting point. Take time to become familiar with the given `needle` source code and pay special attention to the `Op`, `Value`, `Tensor` classes, which can be found in `src/python/needle/autograd.py` and the `Device` class, which can be found in `src/python/needle/device.py`. 

As you saw looking through the needle, we have given the following example operators: element-wise addition and scalar addition, element-wise multiplication, and scalar multiplication. All operators are tracked through the OP_TABLE (found in the `src/python/needle/ops.py` file), and have four key elements: 

1. A class in `src/python/needle/ops.py` that includes a pointer to the forward pass, as well as a function the defines the gradient (or backward pass).  

2. A class in the desired device backend that defines the operation itself (or forward pass). 

3. A method in the Tensor class that allows for the operation to be used on/between instances of Tensor (i.e. the `__add__` method allows two Tensor objects to be added using the `+` symbol). 

4. `OP_TABLE` registration - this must be done for the class created in the `src/python/needle/ops.py` file, as well as the class defined within the device backend. This ensures that when an operator is called on a `Tensor` object, the correct operation is called on the correct device backend. 


## Question 1: Implementing forward computation

In the file `src/python/needle/numpy_backend.py` you will find the forward passes for `add`, `add_scalar`, `multiply` and `multiply_scalar`. Note that the arguments to each of these functions is `inputs` and `attrs` - `inputs` is the list of input (numpy) arrays, whereas `attrs` is a dictionary of additional arguments to the function. In order to see what `inputs` and `attrs` should contain for each function, take a look at the corresponding `Op` classes in `src/python/needle/ops.py`: `EWiseAddOp`, `AddScalarOp`, `EWiseMulOp`, and `EWiseMulScalarOp`. For example, for the case of addition by a scalar, we see that the following function is returned when calling the class `AddScalarOp`: `Tensor.make_from_op(self, [a], attrs={"scalar": scalar})`. 

To start, implement and test (with the following code cell) the forward passes of following operators in `src/python/needle/numpy_backend.py`, taking care to look at the corresponding classes in `src/python/needle/ops.py` for the number of arrays passed in `inputs`, and names of additional arguments in `attrs`. Note that these should all be straightforward, and nearly all will be one line calls to the equivalent `numpy` function. 


- `divide`: true division of the inputs, element-wise
- `divide_scalar`: true division of the input by a scalar, element-wise
- `matmul`: matrix multiplication of the inputs
- `summation`: sum of array elements over given axes
- `broadcast_to`: broadcast an array to a new shape
- `reshape`: gives a new shape to an array without changing its data
- `negate`: numerical negative, element-wise
- `transpose`: reverse or permute the axes of an array; returns the modified array

In [ ]:
!python3 -m pytest -k "forward"

In [ ]:
!python3 -m mugrade submit grader_key -k "forward"

## Question 2: Implementing backward computation

Once you have those working, let's implement the backward passes for those operators. Each class of operator in `src/python/needle/ops.py` has a `gradient` function that takes in three arguments: `output_grads`, `node`, and `attrs`. 

You should implement the backward pass operations using only needle operations on these needle objects. Specifically, fill in the `gradient` function of the following classes:

- `EWiseDivOp`
- `DivScalarOp`
- `MatMulOp`
- `SummationOp`
- `BroadcastToOp`
- `ReshapeOp`
- `NegateOp`
- `TransposeOp`

TODO - need more here

In [ ]:
!python3 -m pytest -k "backward"

In [ ]:
!python3 -m mugrade submit grader_key -k "backward"

## Question 3: Topological sort

Now your system is capable of performing operations on tensors which builds up a computation graph. Now we will write one of the main utilities needed for automatic differentiation - the [topological sort](https://en.wikipedia.org/wiki/Topological_sorting). This will allow us to traverse through (forward or reverse) the compuatation graph, computing gradients along the way. Furthermore, the previously built components will allow for the operations we perform during this reverse topological traversal to further add to our computation graph (as discussed in lecture), and will therefore give us higher-order differentiation "for free." 

Fill out the `find_topo_sort` method and the `topo_sort_dfs` helper method (in `src/python/needle/autograd.py`) to perform this topological sorting. 

TODO - need more here

In [ ]:
!python3 -m pytest -k "topo_sort" -s

In [ ]:
!python3 -m mugrade submit grader_key -k "topo_sort"

## Question 4: Gradient computation

Now that the topological sort is finished, leverage it to calculate the gradients in our computation graph by filling out the `compute_gradient_of_variables` method in `src/python/needle/autograd.py`. This method should walk through the computation graph in reverse order, accumulating gradients along the way. 

TODO - need more here

In [ ]:
!python3 -m pytest -k "compute_gradient"

In [ ]:
!python3 -m mugrade submit grader_key -k "compute_gradient"

## Question 5: Softmax loss

The following questions will be tested using the MNIST dataset, so first copy and paste your solution to Question 2 of Homework 0 to the `parse_mnist` function_ in the `src/python/simple_ml.py` file.  

In this question, you will implement the softmax loss as defined in the `softmax_loss()` function in `src/python/needle/simple_ml.py`, which we defined in Question 3 of Homework 0, except this time, the softmax loss takes as input a `Tensor` of logits and a `Tensor` of one hot encodings of the true labels. 

You will first need to implement the forward and backward passes of two additional operators: ``log`` and ``exp``. Begin by filling out the functions `log`, and `exp` in `src/python/needle/numpy_backend.py` and then fill out the `gradient` function of the classes `LogOp` and `ExpOp` in `src/python/needle/ops.py`. 
 
Once those operators have been implemented, implement the function `softmax_loss` in `src/python/simple_ml.py`. You can start with your solution from Homework 0, and then modify it to be compatible with `needle` objects and operations. 

In [ ]:
!python3 -m pytest -k "softmax_loss_ndl"

In [ ]:
!python3 -m mugrade submit grader_key -k "softmax_loss_ndl"

## Question 6: SGD for a two-layer neural network

As you did in Homework 0, you will now implement stochastic gradient descent (SGD) for a simple two-layer neural network as defined in Question 5 of Homework 0. 

First, you will need to implement the forward and backward passes of the `relu` operator. Begin by filling out the function `relu` in `src/python/needle/numpy_backend.py` and then fill out the `gradient` function of the class `ReLUOp` in `src/python/needle/ops.py`. 

Then, fill out the `nn_epoch` method in the `src/python/simple_ml.py` file. Again, you can use your solution in Homework 0 for the `nn_epoch` function as a starting point. Note that unlike in Homework 0, the inputs `W1` and `W2` are Tensors. Unlike the previous homework, your solution should return the `W1` and `W2` `Tensors`. Inputs `X` and `y` however are still numpy arrays - iterate over minibatches of the numpy arrays `X` and `y` as you did in Homework 0, and then cast `X_batch` as a `Tensor`, and one hot encode `y_batch` and cast as a `Tensor`. Later in the course we will build iterators or data loaders for `Tensor` data. Use the `softmax_loss` you wrote in Question 5, and compute the gradients by calling the `.backward` method of the `Tensor` class.

In [ ]:
!python3 -m pytest -l -k "nn_epoch_ndl"

In [ ]:
!python3 -m mugrade submit YOUR_GRADER_KEY_HERE -k "nn_epoch_ndl"